In [1]:
import torch

import transformers
from transformers import pipeline
from transformers import AutoModelForSequenceClassification, AutoTokenizer

import numpy as np
import pandas as pd

In [ ]:
classifier = pipeline("zero-shot-classification", model="roberta-large-mnli")

In [7]:
sequences_to_classify = ["one day I will see the world", "best steak in america", "FASHION HAUL | Current Favorite Outfits"]
candidate_labels = ['travel', 'cooking', 'clothing', 'exploration']

In [8]:
results = classifier(sequences_to_classify, candidate_labels, multi_label=True)

In [5]:
results

[{'sequence': 'one day I will see the world',
  'labels': ['travel', 'exploration', 'dancing', 'cooking'],
  'scores': [0.9923861026763916,
   0.9649544954299927,
   0.020983165130019188,
   0.013253027573227882]},
 {'sequence': 'best steak in america',
  'labels': ['cooking', 'exploration', 'travel', 'dancing'],
  'scores': [0.9837098121643066,
   0.7697228789329529,
   0.20376132428646088,
   0.0040351515635848045]}]

In [9]:
df = pd.DataFrame(0, index=sequences_to_classify, columns=candidate_labels)

nlabels = len(candidate_labels)
for r in results:
    for i in range(nlabels):
        df.loc[r['sequence'], r['labels'][i]] = r['scores'][i]

df

,travel,cooking,clothing,exploration
one day I will see the world,0.992386,0.013253,0.036708,0.964954
best steak in america,0.203761,0.983710,0.003369,0.769723
FASHION HAUL | Current Favorite Outfits,0.081571,0.000592,0.997936,0.187005


In [ ]:
model_name = "roberta-large-mnli"
tc_pipeline = pipeline("text-classification", model = model_name, top_k=3)

In [56]:
tc_pipeline({"text" : "best steak in america. made with lamb and spices.", "text_pair" : "This example is cooking."})

[{'label': 'ENTAILMENT', 'score': 0.8363721966743469},
 {'label': 'NEUTRAL', 'score': 0.1386958360671997},
 {'label': 'CONTRADICTION', 'score': 0.0249320175498724}]

In [57]:
tc_pipeline({"text" : "best steak in america. made with lamb and spices.", "text_pair" : "This example is clothing."})

[{'label': 'CONTRADICTION', 'score': 0.9754126071929932},
 {'label': 'NEUTRAL', 'score': 0.014543233439326286},
 {'label': 'ENTAILMENT', 'score': 0.010044156573712826}]

In [58]:
tc_pipeline({"text" : "FASHION HAUL | Current Favorite Outfits", "text_pair" : "This example is clothing."})

[{'label': 'ENTAILMENT', 'score': 0.9562535285949707},
 {'label': 'NEUTRAL', 'score': 0.04176858067512512},
 {'label': 'CONTRADICTION', 'score': 0.0019778849091380835}]

In [ ]:
# manual step by step
nli_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [19]:
labels = ['clothing', 'cooking']
hypotheses = [f'This example is {l}.' for l in labels]
sequence = 'best steak in america. made with lamb and spices.'
premises = [sequence] * len(hypotheses)

In [ ]:
tokenizer.tokenize(premises[0], hypotheses[0])

In [21]:
x = tokenizer(premises, hypotheses, return_tensors='pt',
                     truncation='only_first')

In [22]:
x

{'input_ids': tensor([[    0,  7885, 19464,    11, 38187,   102,     4,   156,    19, 17988,
             8, 27235,     4,     2,     2,   713,  1246,    16,  5296,     4,
             2],
        [    0,  7885, 19464,    11, 38187,   102,     4,   156,    19, 17988,
             8, 27235,     4,     2,     2,   713,  1246,    16,  6836,     4,
             2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [23]:
y = nli_model(**x)
logits = y.logits
logits

tensor([[ 2.8108e+00, -1.3950e+00, -1.7651e+00],
        [-1.7178e+00, -1.7096e-03,  1.7951e+00]], grad_fn=<AddmmBackward0>)

In [24]:
# throw away "neutral" (dim 1) and take the probability of
# "entailment" (2) as the probability of the label being true 
e_c_logits = logits[:, [0, 2]]
e_c_logits.softmax(dim=1)

tensor([[0.9898, 0.0102],
        [0.0289, 0.9711]], grad_fn=<SoftmaxBackward0>)

In [25]:
# softmax without throwing away neutral
logits.softmax(dim=1)

tensor([[0.9754, 0.0145, 0.0100],
        [0.0249, 0.1387, 0.8364]], grad_fn=<SoftmaxBackward0>)

In [26]:
nli_model.config

RobertaConfig {
  "_name_or_path": "roberta-large-mnli",
  "_num_labels": 3,
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "CONTRADICTION",
    "1": "NEUTRAL",
    "2": "ENTAILMENT"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "CONTRADICTION": 0,
    "ENTAILMENT": 2,
    "NEUTRAL": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [40]:
x1 = tokenizer("best steak in america. made with lamb and spices. This example is cooking.", return_tensors='pt', truncation='only_first')

In [41]:
x1

{'input_ids': tensor([[    0,  7885, 19464,    11, 38187,   102,     4,   156,    19, 17988,
             8, 27235,     4,   152,  1246,    16,  6836,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [42]:
y1 = nli_model(**x1)
logits1 = y1.logits
logits1

tensor([[-1.5117, -0.0889,  1.6263]], grad_fn=<AddmmBackward0>)

In [44]:
logits1.softmax(dim=1)

tensor([[0.0355, 0.1471, 0.8175]], grad_fn=<SoftmaxBackward0>)

In [52]:
d = {"text" : "best steak in america. made with lamb and spices.", "text_pair" : "This example is cooking."}

'best steak in america. made with lamb and spices.'